In [2]:
#** INFO **
# THIS CODE WAS CREATED FOR USE IN PAPER 'Capturing geological uncertainty in salt cavern developments for hydrogen storage: Case study from Southern North Sea'
# IT IS LICENSED UNDER an CC BY-SA 4.0 
# THIS CODE WAS AIDED IN PARTS BY AI. THE LLM'S USED WERE OPEN AI'S CHAT-GPT4 AND CHAT_GPT 3.5

In [3]:
# NOTE I APOLOGISE FOR THE SPAGHETTI CODE. 
# I APOLOGISE IF ANY OF THE CODING IMPLIMENTATIONS GO AGAINST THE NORM
# IF YOU HVAE ANY TROUBLE USING THIS, GET IN TOUCH AND I'LL HELP - HECTOR BARNETT - H.BARNETT.96@OUTLOOK.COM or c0063859@ncl.ac.uk

In [2]:
import os
import geopandas as gpd
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from rtree import index
from scipy.interpolate import interp1d
from geopandas import GeoSeries
from zmapio import ZMAPGrid
from SALib.sample import saltelli
from SALib.analyze import sobol
from shapely.geometry import Polygon, Point, LineString
from shapely.ops import cascaded_union, unary_union, nearest_points, transform
from shapely.affinity import rotate
from scipy.interpolate import griddata
import pyproj
from pyproj import Transformer
from scipy.spatial.distance import cdist
from scipy.spatial import KDTree, cKDTree
from functools import partial
import progressbar
from tqdm.notebook import tqdm
import folium
from folium import Choropleth
import contextily as ctx
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.colors as mcolors
from matplotlib_scalebar import dimension as si
import shapely.geometry as shp
from math import pi, cos, sin, sqrt
from concurrent.futures import ProcessPoolExecutor
from joblib import Parallel, delayed
from scipy.constants import g
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import truncnorm


# Functions needed 

In [3]:
# FUNCTIONS USED GET LOADED IN

In [4]:
#Overburden pressure functions
#Zmap functions

def read_zmap_to_geopandas(zmap_filename, crs='EPSG:32631'):
    # Open the zmap+ file and read the z values into a pandas dataframe. So technically need to do nearest neighbor for x values
    zgrid = ZMAPGrid(zmap_filename)
    zdf = zgrid.to_pandas() # this basically puts the zmap into a pandas df
    zdf2 = gpd.GeoDataFrame(zdf, geometry=gpd.points_from_xy(zdf['X'], zdf['Y'], zdf['Z']), crs='EPSG:32631') #this converts the pandas df into a geopandas df and hopefully gives the cprrect CRS
    zdf3 = zdf2.dropna(subset=['Z'])
    
    

    return zdf3

def add_column(df: gpd.GeoDataFrame, gpd_B: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    # Get the nearest neighbor from gpd_B for each point in df
    nA = np.array(list(df.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gpd_B.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gpd_B.iloc[idx].reset_index(drop=True)

    # Concatenate columns from gdB_nearest whose names contain 'Z_'
    z_cols = [col for col in gdB_nearest.columns if 'Z_' in col]
    result = pd.concat([df.reset_index(drop=True), gdB_nearest[z_cols]], axis=1)

    # Return the result as a new GeoDataFrame
    return gpd.GeoDataFrame(result, geometry='geometry')


def attach(l1,l2,l3,l4,l5):
    T1 = add_column(l1,l2)
    T2 = add_column(T1,l3)
    T3 = add_column(T2,l4)
    T4 = add_column(T3,l5)
    
    return T4


In [5]:
def attach(l1,l2,l3,l4,l5):
    T1 = add_column(l1,l2)
    T2 = add_column(T1,l3)
    T3 = add_column(T2,l4)
    T4 = add_column(T3,l5)
    
    return T4

In [6]:
#converts CRS
def convert_crs_to_epsg_32631(gdf):
    """Converts the CRS of a GeoDataFrame to EPSG:32631"""
    gdf_32631 = gdf.to_crs(epsg=32631)
    return gdf_32631

In [7]:
# Cleans input shapefiles from petrel and makes them usable
def linear_to_polygon(df):
    # Loop through all geometries in the GeoDataFrame
    for idx, geom in enumerate(df.geometry):
        # Check if the geometry is a LineString or MultiLineString
        if geom.geom_type in ['LineString', 'MultiLineString']:
            # Check if the geometry is closed, and close it if necessary
            if not geom.is_closed:
                geom = geom.union(geom[0])
            # Create a Polygon from the linear geometry
            polygon = Polygon(geom)
            # Replace the linear geometry with the Polygon in the GeoDataFrame
            df.loc[idx, 'geometry'] = polygon
    
    # Drop all attributes (columns) from the GeoDataFrame
    df = df.loc[:, ['geometry']]
    
    # Add an ID column to the GeoDataFrame
    df['id'] = range(1, len(df)+1)

    df.geometry = convert_3D_2D(df.geometry) # new geodf with 2D geometry series
    
    df = convert_crs_to_epsg_32631(df)
    
    return df



#data cleaning so geopandas is happy, converts 3d shapefile to 2d........why are they 3d to begin with? idk ask petrel
def convert_3D_2D(geometry):
    '''
    Takes a GeoSeries of 3D Multi/Polygons (has_z) and returns a list of 2D Multi/Polygons
    '''
    new_geo = []
    for p in geometry:
        if p.has_z:
            if p.geom_type == 'Polygon':
                lines = [xy[:2] for xy in list(p.exterior.coords)]
                new_p = Polygon(lines)
                new_geo.append(new_p)
            elif p.geom_type == 'MultiPolygon':
                new_multi_p = []
                for ap in p:
                    lines = [xy[:2] for xy in list(ap.exterior.coords)]
                    new_p = Polygon(lines)
                    new_multi_p.append(new_p)
                new_geo.append(MultiPolygon(new_multi_p))
        else:
            new_geo.append(p)  # Append the 2D geometry as-is

    return new_geo


def convert_linestringZ_to_linestring(df):
    # Check if the CRS is already EPSG:32631
    if df.crs is None or df.crs != 'epsg:32631':
        # If not, convert the CRS to EPSG:32631
        df = df.to_crs(epsg=32631)

    # Create a new GeoDataFrame with only the "geometry" and "type" columns
    new_df = gpd.GeoDataFrame(geometry=df.geometry, crs='epsg:32631')

    # Iterate over the rows in the output DataFrame
    for index, row in new_df.iterrows():
        # Convert the linestring Z geometry to a plain linestring
        coords = row['geometry'].coords
        new_coords = [(coord[0], coord[1]) for coord in coords]
        new_geometry = LineString(new_coords)

        # Update the "geometry" column with the converted geometry
        new_df.at[index, 'geometry'] = new_geometry

    # Return the output DataFrame
    return new_df

In [8]:
#makes outline of suitable cavern zone, e'g caverns can go within generated shapefile/gpd

def make_suitable_Cavern_Zone(faults, faultsbuffer, HC_Fields, depo_feat, siteoutline, salt_thick=None, salt_depth=None):
    # copy gpd so things don't get messy
    buffer = siteoutline.copy()
    faults = faults.copy()
    depo_feat = depo_feat.copy()
    HC_Fields = HC_Fields.copy()
    if salt_thick is not None:
        salt_thick = salt_thick.copy()
    if salt_depth is not None:
        salt_depth = salt_depth.copy()
    
    # reset indices of GeoSeries
    buffer = buffer.reset_index(drop=True)
    faults = faults.reset_index(drop=True)
    HC_Fields = HC_Fields.reset_index(drop=True)
    depo_feat = depo_feat.reset_index(drop=True)
    if salt_thick is not None:
        salt_thick = salt_thick.reset_index(drop=True)
    if salt_depth is not None:
        salt_depth = salt_depth.reset_index(drop=True)
    
    if salt_thick is not None:
        salt_thick = salt_thick.dissolve(by=None)
    HC_Fields = HC_Fields.dissolve(by=None)
    depo_feat = depo_feat.dissolve(by=None)
    if salt_depth is not None:
        salt_depth = salt_depth.dissolve(by=None)

    # creates buffer around fault sticks, user input
    Faults_area = create_buffer_and_dissolve(faults, faultsbuffer)
    
    diff1 = buffer.difference(Faults_area)
    diff2 = diff1.difference(HC_Fields)
    diff3 = diff2.difference(depo_feat)
    diff4 = diff3
    if salt_thick is not None:
        diff4 = diff3.difference(salt_thick)
    diff5 = diff4
    if salt_depth is not None:
        diff5 = diff4.difference(salt_depth)
    
    diff6 = gpd.GeoDataFrame(geometry=diff5)
    mask = gpd.clip(diff6, siteoutline)
    
    return mask





# function used in above
def create_buffer_and_dissolve(input_gpd, buffer_distance):
    # Create a copy of the input GeoDataFrame
    input_gpd_copy = input_gpd.copy()

    # Check if the input GeoPandas DataFrame has LineString geometries
    if not all(input_gpd_copy.geometry.type == "LineString"):
        raise ValueError("The input GeoDataFrame must contain only LineString geometries.")

    # Create a buffer around each LineString
    input_gpd_copy['buffer'] = input_gpd_copy.geometry.buffer(buffer_distance)

    # Set the new 'buffer' column as the active geometry
    input_gpd_copy.set_geometry('buffer', inplace=True)

    # Dissolve overlapping buffer polygons
    dissolved_gpd = input_gpd_copy.dissolve(by=None)

    return dissolved_gpd

In [9]:
# appends points to nearest points..... this one is used for Depth and salt thickness, as strings are labelled

def nearestneighbor(df: gpd.GeoDataFrame, gpd_B: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    nA = np.array(list(df.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gpd_B.geometry.apply(lambda x: (x.x, x.y))))

    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)

    gdB_nearest = gpd_B.iloc[idx].reset_index(drop=True)

    gdf = pd.concat(
        [
            df.reset_index(drop=True),
            gdB_nearest["Depth"],
            pd.Series(dist, name='dist')
        ],
        axis=1
    )

    return gdf



def nearestneighbor_OBP(df: gpd.GeoDataFrame, gpd_B: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    nA = np.array(list(df.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gpd_B.geometry.apply(lambda x: (x.x, x.y))))

    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)

    gdB_nearest = gpd_B.iloc[idx].reset_index(drop=True)

    gdf = pd.concat(
        [
            df.reset_index(drop=True),
            gdB_nearest["obp"],
            pd.Series(dist, name='dist')
        ],
        axis=1
    )

    return gdf

In [10]:
def nearestneighbor_OBP(df: gpd.GeoDataFrame, gpd_B: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    nA = np.array(list(df.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gpd_B.geometry.apply(lambda x: (x.x, x.y))))

    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)

    gdB_nearest = gpd_B.iloc[idx].reset_index(drop=True)

    gdf = pd.concat(
        [
            df.reset_index(drop=True),
            gdB_nearest[["obp@topZ2", "Z_Topstass"]],
            pd.Series(dist, name='dist')
        ],
        axis=1
    )

    return gdf


In [11]:
def add_cavern_data(df):
    df = df.rename(columns={'Z': 'thickness'})
    df['Base'] = df['thickness'] + df['Depth']
    df['thick_avail'] = df['thickness']
    df['cav_top'] = df['Depth'].apply(lambda x: x if x >= 500 else 500)
    df['thick_avail'] = df['Base'] - df['cav_top']    
    df = df[df['thick_avail'] >= 200]

    #Uncomment for fixed caverns
    #df = df[df['thick_avail'] >= 358.5] # change for varible cavern runs to 200. 358.5 for varible caverns    
    #df = df[df['Base'] - df['cav_top'] >= 200] #makes sure still 20m thickness
    #df = df[df['thick_avail'] >= 358.5] for fixed caverns
    
    
    df['Extra_hanging_wall'] = df['cav_top'] - df['Depth']
    

    
    
    ###### BELOW IS FOR VARIBLE CAVERNS ######
    df = df.rename(columns={'thickness': 'Z'})
    df = calculate_height_to_diameter_ratio(df)
    df = df.rename(columns={'Z': 'thickness'})
    df['max_cavern_height'] = df['thick_avail'] / (1 + 0.95 / df['height_to_diameter_ratio'])
    df.loc[df['max_cavern_height'] > 750, 'max_cavern_height'] = 750
    df['cavern_diameter'] = df['max_cavern_height'] / df['height_to_diameter_ratio']
    ###############################################
     
    
    ######### BELOW IS USED Fixed Caverns###### UNCOMMENT IF YOU WANT FIXED CAVERNS, AND CHANGE TO DESIRED CAVERN PARAMS
    #df['cavern_diameter'] = 58
    #df['max_cavern_height'] = 300
    #################################################################
    
    
    
    
    
    df['height_to_diameter_ratio'] = df['max_cavern_height'] / df['cavern_diameter']
    
    
    df['footwall'] = 0.75 * df['cavern_diameter']
    df['hangingwall'] = 0.2 * df['cavern_diameter']
    df['hangingwall'] = df['hangingwall'] + df['Extra_hanging_wall']
    df['heightcheck'] = df['max_cavern_height'] + df['footwall'] + df['hangingwall']
    df['is_valid'] = df['heightcheck'] <= df['thick_avail']
    
    
    
    df['buffer'] = 2 * df['cavern_diameter']
    
    df['total_cavern_height'] = df['max_cavern_height'] +  df['footwall'] + df['hangingwall']
    
    df['cav_top_depth'] = df['cav_top'] + df['hangingwall'] + df['Extra_hanging_wall']
    df.loc[pd.isna(df['cav_top_depth']), 'cav_top_depth'] = df['Depth'] + df['hangingwall']
    
    
    df['cav_mid_depth'] = df['cav_top_depth'] + (0.5 * df['max_cavern_height']) #returns mid point of cavern in z
    
    
    
    #df['cav_vol'] = 4/3 * (math.pi) * (df['cavern_diameter']/2) * (df['cavern_diameter']/2) * (df['max_cavern_height']/2)
    df['cav_vol'] = df.apply(lambda row: calculate_capsule_or_cylinder_volume(row['max_cavern_height'], row['cavern_diameter']), axis=1)
    # remove HDR from the dataframe if you no longer need it
    #df = df.drop('height_to_diameter_ratio', axis=1)
    
    return df


In [12]:
def calculate_capsule_or_cylinder_volume(height, diameter):
    radius = diameter / 2
    cylindrical_height = height - 2 * radius
    
     #Check if the shape is a cylinder
    if cylindrical_height < 0.1:
        volume = 4/3 * (math.pi) * (diameter/2) * (diameter/2) * (height/2)
    else:
        volume = (math.pi * radius ** 2 * cylindrical_height) + (4 / 3) * math.pi * radius ** 3  # Volume of a capsule
    
    return volume

In [13]:
def calculate_capsule_or_cylinder_volume(height, diameter):
    radius = diameter / 2
    cylindrical_height = height - 2 * radius
    
     #Check if the shape is a cylinder
    if cylindrical_height < 0.1:
        volume = 4/3 * (math.pi) * (diameter/2) * (diameter/2) * (height/2)
    else:
        volume = (math.pi * radius ** 2 * cylindrical_height) + (4 / 3) * math.pi * radius ** 3  # Volume of a capsule
    
    return volume


In [14]:
def hydrogen_values(df,Soluble_samples1,gfg3):
    
    #You will need to modify accordingly for basin of interest 
    seabedtemp = 12
    df['cav_mid_to_sb'] = df['cav_mid_depth'] - df['SB_Z']
    
    #USE BELOW FOR ALL CAVERNS TO HAVE OWN RANDOM GFG 
    #df['gtg'] = gfg3
    
    #USE BELOW FOR WHOLE AREA TO HAVE GFG FROM DISTRABUTUION
    random_number = np.random.choice(gfg3)
    df['gtg'] = random_number
    
    
    #COMMENT FOR NONE CASE STUDY 1
    #df['cav_top_depth'] = df['Z_Topstass'] + df['hangingwall']
    #df['cav_mid_depth'] = df['Z_Topstass'] + df['hangingwall'] + (0.5 * df['max_cavern_height']) #returns mid point of cavern in z
    
  
   
   
   #UNCOMMENT BELOW FOR GENERIC BASIN CASE STUDY
   #df['cav_top_depth'] = df['Depth'] + df['hangingwall']
  
    
   # df['cav_mid_depth'] = df['Depth'] + df['hangingwall'] + (0.5 * df['max_cavern_height']) #returns mid point of cavern in z
    df['cav_mid_temp'] = seabedtemp + (df['gtg'] * (df['cav_mid_to_sb']/1000))  # finds mid temp of cavern in c using above anf geothermal gradien
    
    df['insolubles%'] = Soluble_samples1
    
    df['act_cav_vol'] = df['cav_vol'] * df['insolubles%']
    

    
    
    
    ###     UNCOMMENT FOR  Gradient / Layercake OBP   ###### is currently using a gradient of 0.023544
    
    
    
    #df['obp4z2'] = (df['hangingwall'] * g * halite)/1000000
    #df['obp'] = df['obp4z2'] + df['obp@topZ2']
    
    #OBP calc for whole SNS BASIN case study 1 assumes 2040 overburden, 0.01 is seawater density
    df['obp'] = (df['cav_mid_to_sb'] * 0.023544) + (df['SB_Z'] * 0.01)
    
    #OBP calc for structuresa assumes halite at a density of 2040kg
    #df['obp'] = (df['cav_mid_to_sb'] * 0.023544) + (df['SB_Z'] * 0.01)
    
    
    

    
    df['Max_Theore_Cap_H_n'] = (df['act_cav_vol'] * (((df['obp'] *1000000)*.6))) / (8.31*(df['cav_mid_temp'] + 273)) #Ideal gas law. Have to conver Mpa to Kpa and C to K, 0.6 as 20% cushion and 0.8 max so 0.6 is working gas
    df['Max_Theoret_Cap_H_Mass_Kg'] = (df['Max_Theore_Cap_H_n'] * 2.016) / 1000 #KG value of hydrogen
    df['H2_Mass_ton_TM'] = (df['Max_Theoret_Cap_H_Mass_Kg']) / 1000 #Ton value of hydrogen
    df['H2_E_GWh_TM'] = ((df['Max_Theoret_Cap_H_Mass_Kg']) * 119.96) / 3.6e+6 # This conversion value taken from 2022 BGS paper, divide by 3.6e+6 for mj to GwH
    #df['H2_E_GWh_GPT'] = df['Max_Theoret_Cap_H_Mass_Kg'] * 141.86 / 1000000 # can remove if unneeded/diff to other value
    df = df.drop(columns =['Max_Theore_Cap_H_n','Max_Theoret_Cap_H_Mass_Kg'])
    #Total = df['H2_E_GWh_TM'].sum()
    
    #print(Total, 'GwH')
    return df




In [15]:
def generate_circles_within_polygon(gpd_A, gpd_B):
    gpd_B_polygon = gpd_B.unary_union
    circles = []
    attributes = []

    for idx, row in gpd_A.iterrows():
        point, buffer = row['geometry'], row['buffer']
        circle = point.buffer(buffer)

        if gpd_B_polygon.contains(circle):
            circles.append(circle)
            attributes.append(row.drop('geometry'))

    non_overlapping_circles = []
    non_overlapping_attributes = []
    for circle, attr in zip(circles, attributes):
        if not any(circle.intersects(existing_circle) for existing_circle in non_overlapping_circles):
            non_overlapping_circles.append(circle)
            non_overlapping_attributes.append(attr)

    new_gpd = gpd.GeoDataFrame(non_overlapping_attributes, geometry=non_overlapping_circles, crs=gpd_A.crs)
    return new_gpd

In [16]:
def generate_circles_within_polygon(gpd_A, gpd_B):
    # Create a unary union of all geometries in gpd_B
    gpd_B_polygon = gpd_B.unary_union
    
    # Initialize R-tree index
    spatial_idx = index.Index()
    
    # Initialize empty lists to store non-overlapping circles and their attributes
    non_overlapping_circles = []
    non_overlapping_attributes = []
    
    # Loop through each row in gpd_A to create circles
    for idx, row in gpd_A.iterrows():
        point, buffer = row['geometry'], row['buffer']
        
        # Create a circle (buffer) around the point
        circle = point.buffer(buffer)
        
        # Check if the circle is within the gpd_B_polygon
        if gpd_B_polygon.contains(circle):
            
            # Get intersecting circles' indices using R-tree index
            intersecting_circles = list(spatial_idx.intersection(circle.bounds))
            
            # Check if circle intersects with any of the existing non-overlapping circles
            if not any(non_overlapping_circles[i].intersects(circle) for i in intersecting_circles):
                
                # If not, add the circle and its attributes to the lists
                non_overlapping_circles.append(circle)
                non_overlapping_attributes.append(row.drop('geometry'))
                
                # Insert the circle into R-tree index
                spatial_idx.insert(len(non_overlapping_circles) - 1, circle.bounds)

    # Create a new GeoDataFrame with non-overlapping circles and attributes
    new_gpd = gpd.GeoDataFrame(non_overlapping_attributes, geometry=non_overlapping_circles, crs=gpd_A.crs)
    
    return new_gpd


In [17]:
def create_circle_gdf(gdf, buffer_col):
    # copy the dataframe to keep original data
    gdf_copy = gdf.copy()
    # create circles using buffer
    gdf_copy['geometry'] = gdf_copy['geometry'].buffer(gdf_copy[buffer_col])
    return gdf_copy

In [18]:
def convert_circles_to_centroids(df):
    df['centroid'] = df['geometry'].centroid
    df['centroid_x'] = df['centroid'].x
    df['centroid_y'] = df['centroid'].y
    centroids_df = gpd.GeoDataFrame(df.drop('geometry', axis=1), geometry=df['centroid'])
    centroids_df = centroids_df.drop(['centroid', 'centroid_x', 'centroid_y'], axis=1)
    return centroids_df

In [19]:
import random

def generate_random_percentage(uncertainty):
    lower_bound = 1 - uncertainty / 100
    upper_bound = 1 + uncertainty / 100
    return random.uniform(lower_bound, upper_bound)


In [20]:
i = generate_random_percentage(7)

In [79]:
def process_salt_data(Salt_Thick, Salt_Depth, Suitable_Zone_10, params, depth_uncert2):
    # Extract the necessary parameters from the params dictionary
    hdr_base_loc = params.get('hdr_base_loc')
    hdr_scale = params.get('hdr_scale')
    hdr_loc_factor = params.get('hdr_loc_factor')
    hdr_min = params.get('hdr_min')
    hdr_max = params.get('hdr_max')
    insoluble_mean = params.get('insol_mean')
    insoluble_std_dev = params.get('insol_std_dev')
    geotherm_mean = params.get('geotherm_mean')
    geotherm_std_dev = params.get('geotherm_std_dev')
    BATHY = params.get('BATHY')
    
    Salt_Depth1 = Salt_Depth.copy()
            
            
    Salt_Thick1 = Salt_Thick.copy()
    
    depth_uncert2 = depth_uncert2
    
    
    
    Salt_Thick1['Z'] = Salt_Thick1['Z'] * depth_uncert2
    Salt_Depth1['Depth'] = Salt_Depth1['Depth'] * depth_uncert2
    

    Salt_Stage1 = nearestneighbor(Salt_Thick1, Salt_Depth1)
    Salt_Stage1 = Salt_Stage1.drop('dist', axis=1)
    Salt_Stage1['depth_uncert'] = depth_uncert2
    print(Salt_Stage1)
    print('Doing bathy')
    #FIXGEOTHERMAL
    Salt_Stage1 = nearestneighbor_bath(Salt_Stage1, BATHY)
    print('Bathy_Done')
    Salt_Stage1 = Salt_Stage1.drop('dist', axis=1) 
    Salt_Stage1



    
    #########CHANGE BACK TO 200M FOR DEFAULT 300m -----> 200m
    Salt_Stage1 = Salt_Stage1[Salt_Stage1['Z'] >= 200]
    Salt_Stage1 = Salt_Stage1[Salt_Stage1['Depth'] <= 1700]
    #HDR_Values = generate_HDR_Values(Salt_Stage1, params)
    #########CHANGE BACK TO 200M FOR DEFAULT 300m -----> 200m
    
    #Salt_Stage1 = Salt_Stage1[Salt_Stage1['Z'] >= 358.5]
    #Salt_Stage1 = Salt_Stage1[Salt_Stage1['Depth'] <= 1700]
    
    
    
     #Salt_Stage11 = calculate_height_to_diameter_ratio(Salt_Stage1)   #replaced in below function
    Salt_Stage2 = add_cavern_data(Salt_Stage1)
    print('Stage 2 done')
    print(Salt_Stage2)
    print('Stage 2 above df')
    
    

    Salt_Stage3 = generate_circles_within_polygon(Salt_Stage2, Suitable_Zone_10)
    print(Salt_Stage3)
 
    print('Stage 3 done')
    
    print(Salt_Stage3)
    print('Stage 3 df above')

    Salt_Stage4 = convert_circles_to_centroids(Salt_Stage3)
    print('Stage 4 done')

    return Salt_Stage4

def process_cavern_data(processed_salt_data, params, OBP=None):     
        
    
    #Uncomment below if using layer cake OBP model
    
    #Salt_Stage5 = nearestneighbor_OBP(processed_salt_data, OBP)
    
    #will have to comment line below if so
    
  
    Salt_Stage5 = processed_salt_data 
    #Salt_Stage5.drop(['dist'], axis=1, inplace=True)
    #print('Stage 5 done')

    
   # print('Stage 6 done')
   
    #Code here to define insoluble values
    
    Soluble_samples = generate_insoluble_Values(Salt_Stage5, params)
    
    Geotherm = generate_Geotherm_Values(Salt_Stage5, params)
    
    
    #Salt_Stage6 = hydrogen_values(Salt_Stage5, Soluble_samples, Geotherm)
    # remove when not doing structure
    Salt_Stage6 = hydrogen_values(Salt_Stage5, Soluble_samples, Geotherm)
    
    print('Stage 7 done')

    Stage_7 = create_circle_gdf(Salt_Stage6, 'buffer')
    
    Stage_7.drop(['is_valid', 'heightcheck'], axis=1, inplace=True)
    
    return Stage_7




In [22]:
import numpy as np

def calculate_overburden_pressure(df_layers, params,depth_uncert1):
    ## Extract the necessary parameters from the params dictionary
    water_den_mean = params.get('water_den_mean')
    tert_den_mean = params.get('tert_den_mean')
    bunter_den_mean = params.get('bunter_den_mean')
    Zech_top_mean = params.get('Zech_top_mean')
    halite_mean = params.get('halite_mean')
    hanging_wall_thick_mean = params.get('hanging_wall_thick_mean')
    
    depth_uncert = depth_uncert1
    
    

    OBP = df_layers.copy()
    
    columns_to_multiply = ['Z_Datum', 'Z_Seabed', 'Z_Bunter', 'Z_TopZechob', 'Z_Topstass']

    for column in columns_to_multiply:
        if column in OBP.columns:
            OBP[column] = OBP[column] * depth_uncert

    # Generate possible values
    water_den = np.random.uniform(water_den_mean * 1, water_den_mean * 1)
    tert_den = np.random.uniform(tert_den_mean * 0.9, tert_den_mean * 1.1)
    bunter_den = np.random.uniform(bunter_den_mean * 0.9, bunter_den_mean * 1.1)
    Zech_top = np.random.uniform(Zech_top_mean * 0.9, Zech_top_mean * 1.1)
    halite = np.random.uniform(halite_mean * 0.9, halite_mean * 1.1)
    hanging_wall_thick = np.random.uniform(hanging_wall_thick_mean * 0.9, hanging_wall_thick_mean * 1.1)
    
    # Assume gravitational constant g in m/s²
    g = 9.81

    # Calculate thickness
    OBP['w_thick'] = abs(OBP['Z_Seabed'] - OBP['Z_Datum'])
    OBP['tert_thick'] = abs(OBP['Z_Bunter'] - OBP['Z_Seabed'])
    OBP['bunter_thick'] = abs(OBP['Z_TopZechob'] - OBP['Z_Bunter'])
    OBP['tz_thick'] = abs(OBP['Z_Topstass'] - OBP['Z_TopZechob'])
    
    #######Between hash added for BGS data. REMOVE AFTER USE!
    OBP['Tot_thick'] = OBP['w_thick'] + OBP['tert_thick'] + OBP['bunter_thick'] +  OBP['tz_thick']
    OBP['obp'] = (OBP['Tot_thick'] * g * 2400) / 1000000
    OBP['obp@topZ2'] = (OBP['Tot_thick'] * g * 2400) / 1000000
    #######
    
    # Calculate overburden pressures for each layer
    OBP['w_obp'] = (OBP['w_thick'] * g * water_den) / 1000000
    OBP['tert_obp'] = (OBP['tert_thick'] * g * tert_den) / 1000000
    OBP['bunter_obp'] = (OBP['bunter_thick'] * g * bunter_den) / 1000000
    OBP['tz_obp'] = (OBP['tz_thick'] * g * Zech_top) / 1000000
    
    # Calculate total overburden pressure###### UNDO
    OBP['obp'] = OBP['w_obp'] + OBP['tert_obp'] + OBP['bunter_obp'] + OBP['tz_obp']
    OBP['obp@topZ2'] = OBP['w_obp'] + OBP['tert_obp'] + OBP['bunter_obp'] + OBP['tz_obp']
    
    # Drop the columns you no longer need
    OBP.drop(['Z_Datum','Z_Seabed','Z_Bunter','Z_TopZechob','w_obp','tert_obp','bunter_obp','tz_obp'], axis=1, inplace=True)###
    
    return OBP
    


In [23]:
def nearestneighbor_obp2(df: gpd.GeoDataFrame, gpd_B: gpd.GeoDataFrame, Z_name: str) -> gpd.GeoDataFrame:
    nA = np.array(list(df.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gpd_B.geometry.apply(lambda x: (x.x, x.y))))

    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)

    gdB_nearest = gpd_B.iloc[idx].reset_index(drop=True)

    gdf = pd.concat(
        [
            df.reset_index(drop=True),
            gdB_nearest[Z_name]
        ],
        axis=1
    )

    return gdf


In [24]:

def calculate_grid_spacing(gdf):
    # Assuming 'X' and 'Y' are your coordinate columns
    unique_x = np.unique(gdf['X'])
    unique_y = np.unique(gdf['Y'])
    
    if len(unique_x) < 2 or len(unique_y) < 2:
        print('Not enough unique X or Y values to calculate grid spacing.')
        return
    
    # Compute grid spacing
    grid_spacing_x = unique_x[1] - unique_x[0]
    grid_spacing_y = unique_y[1] - unique_y[0]

    print('Grid spacing in X:', grid_spacing_x)
    print('Grid spacing in Y:', grid_spacing_y)


In [25]:
def resample_grid(df, x_spacing, y_spacing):
    # Calculate the number of points needed along the X and Y axes based on the desired spacing
    x_points = int((df['X'].max() - df['X'].min()) / x_spacing)
    y_points = int((df['Y'].max() - df['Y'].min()) / y_spacing)

    # Create a grid of points
    xi = np.linspace(df['X'].min(), df['X'].max(), x_points)
    yi = np.linspace(df['Y'].min(), df['Y'].max(), y_points)
    xi, yi = np.meshgrid(xi, yi)

    # Use scipy's griddata function to interpolate the Z values onto the new grid
    zi = griddata((df['X'], df['Y']), df['Z'], (xi, yi), method='linear')

    # Flatten the grid to a list of points
    x = xi.flatten()
    y = yi.flatten()
    z = zi.flatten()

    # Create a new DataFrame
    data = {'X': x, 'Y': y, 'Z': z, 'geometry': [Point(x[i], y[i], z[i]) for i in range(len(x))]}
    resampled_df = pd.DataFrame(data)

    # Convert the DataFrame to a GeoDataFrame
    resampled_gdf = gpd.GeoDataFrame(resampled_df, geometry='geometry')
    
    
    resampled_gdf = resampled_gdf.dropna(subset=['Z'])

    resampled_gdf = resampled_gdf.set_crs(epsg=32631)
    
    return resampled_gdf

In [26]:
def nearestneighbor_bath(df: gpd.GeoDataFrame, gpd_B: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    nA = np.array(list(df.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gpd_B.geometry.apply(lambda x: (x.x, x.y))))

    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)

    gdB_nearest = gpd_B.iloc[idx].reset_index(drop=True)

    gdf = pd.concat(
        [
            df.reset_index(drop=True),
            gdB_nearest["SB_Z"], #was depth
            pd.Series(dist, name='dist')
        ],
        axis=1
    )

    return gdf

In [27]:
def combined_pro(Salt_Thick, Salt_Depth, Suitable_Zone_10, df_layers, params, depth_uncert1):
    
    
    depth_uncert2 = generate_random_percentage(depth_uncert1)
    # rremove line below for montecarlo
    #depth_uncert2 = depth_uncert1 #this line has now been removed
    print(depth_uncert2)
    P1 = process_salt_data(Salt_Thick, Salt_Depth, Suitable_Zone_10, params, depth_uncert2)

    P2 = process_cavern_data(P1,  params, OBP=None)

    return P2


## Functions for paramaters. Insolubles, geothemal, height to diameter ratios

In [28]:
def generate_insoluble_Values(df, params, seed=None):
    
    insoluble_mean = params.get('insol_mean')
    insoluble_std_dev = params.get('insol_std_dev')
    
    # Define the truncation limits
    lower_limit = -np.inf  # no lower limit
    upper_limit = 100  # upper limit is 100

    # Convert the limits to the standard normal scale
    lower_limit_standard = (lower_limit - insoluble_mean) / insoluble_std_dev
    upper_limit_standard = (upper_limit - insoluble_mean) / insoluble_std_dev

    # Create a specific random state
    if seed is not None:
        rng = np.random.default_rng(seed)
    else:
        rng = np.random.default_rng()

    # Number of samples to generate
    num_samples = 1000

    # Generate random samples from the truncated normal distribution
    Soluble_samples = truncnorm.rvs(lower_limit_standard, upper_limit_standard, loc=insoluble_mean, scale=insoluble_std_dev, size=num_samples, random_state=rng)

    # Assuming the division operation is element-wise and the denominator is not zero
    # If your denominator is a variable, replace 'denominator' with your actual variable
    denominator = 1  # Replace with actual value
    if denominator != 0:
        Soluble_samples = Soluble_samples / denominator
    else:
        raise ZeroDivisionError("Denominator cannot be zero.")

    # Sample random values from Soluble_samples to match the size of the input DataFrame
    samples = rng.choice(Soluble_samples, size=len(df), replace=True)
    
    
    samples = samples/100

    return samples


In [29]:

def generate_Geotherm_Values(df, params):
    
    geotherm_low = params.get('geotherm_low')  # Lower bound of the uniform distribution
    geotherm_high = params.get('geotherm_high')  # Upper bound of the uniform distribution

    # Number of samples to generate
    num_samples = 1000

    # Generate random samples from the uniform distribution
    Geotherm_samples = np.random.uniform(low=geotherm_low, high=geotherm_high, size=num_samples)
    
    # Sample random values from Geotherm_samples to match the size of the input DataFrame
    samples = np.random.choice(Geotherm_samples, size=len(df), replace=True)

    return samples


In [30]:


    def calculate_height_to_diameter_ratio(df):
        # Given data for interpolation
        height_to_diameter_ratio = [7.5, 7.5, 3, 1.8, 0.8, 0]
        depth = [5000, 750, 500, 400, 200, 0]
        
        # Create an interpolation function
        interpolation_function = interp1d(depth, height_to_diameter_ratio, kind='linear', fill_value="extrapolate")
        
        # Calculate the height-to-diameter ratio based on the interpolated function
        df['height_to_diameter_ratio'] = df['Z'].apply(interpolation_function)
        
        return df

# Load Varibles / minor varible manipulation

In [61]:
#Parameters to load in. Surfaces should be in the form of ZMAPS. 
#####run petrel exported shapefiles thorugh this, unless faults linear_to_polygon

#This is the Depth surface varible
#Example for surface taken out of petrel as a zmap
Salt_Depth = read_zmap_to_geopandas('X:\Datarepo\Inputs Geo Models\Block specific\Layered Evaporite\example for code\TopZ2.ascii')
Salt_Depth = Salt_Depth.rename(columns={'Z': 'Depth'})
Salt_Depth = Salt_Depth.drop(columns=["X", "Y"])


# This is the thickness map of salt varible, an alternate way to do this would be to have a depth surface. Having this pre-made saves calulcation time which can add several hours onto large
Salt_Thick =  read_zmap_to_geopandas('X:\Datarepo\Inputs Geo Models\Block specific\Layered Evaporite\example for code\Z2Thick.ascii')

In [56]:
####infrastructure or other areas of exclusions sites####

#Ignore data for filling varibles if they data is not available, se ebelow
data = {'X': [0]} #Filling code if wanted exclusion is not available
dataxy = {'X': [0], 'Y' : [1]}
line = LineString([(0, 0), (dataxy['X'][0], dataxy['Y'][0])])
######

#Site1 is the  area outline for the cavern placement you NEED this shapfile. # Load in from the data repo to have a look. You can examine by typing 
#Site1 = linear_to_polygon(gpd.read_file('X:\Datarepo\Inputs Geo Models\Block specific\Layered Evaporite\Area_Outline.gpkg'))  # Site 1 is just the area outline




#This are the exclusion sites, the uncommented here assumes you have the data (if you're trying from the data repo). You can swap the uncomments over and run from having no data 
HC_Fields = convert_crs_to_epsg_32631(gpd.read_file('X:/TAKE FROM NSTA WEBSITE'))
#HC_Fields = gpd.GeoDataFrame(data, geometry=[Point(0, 0)])
Evap_depo_feat = linear_to_polygon(gpd.read_file('X:\Datarepo\Inputs Geo Models\Block specific\Layered Evaporite\Exclusion Zones\Unviable depositional Features Areas\depo_feat.shp')) # this is area of unviable salt, could map any features here
#Evap_depo_feat = gpd.GeoDataFrame(data, geometry=[Point(0, 0)])
Faults = convert_linestringZ_to_linestring(gpd.read_file('X:\Datarepo\Inputs Geo Models\Block specific\Layered Evaporite\Exclusion Zones\Faults\Faults.shp'))
#Faults = gpd.GeoDataFrame(dataxy, geometry=[line])

In [ ]:
#Examine the geospaital files below
#Site.explore()

In [33]:
### TOO CHANGE CAVERN SIZE FROM VARIBLE TO FIXED ###
# Go to function 'add_cavern_data' and comment / uncomment as instructed, this will allow for changing from varible to fixed
# Apologies for the odd way of doing this, maybe i'll get round to automating this when i have time, but it works currently so........

In [34]:
#LOAD IN BATHYMETRY, i guess could just set to 0 if easier see cell below
# this assumes that the rastor bathymetry data has been converted into vector points
# recommend getting bathymetry data from emodnet

#BATHY = gpd.read_file('X:\Data\Bathymetery\Point_Bath_DS.gpkg')
#BATHY.rename(columns={'Z': 'SB_Z'}, inplace=True)
#BATHY = BATHY.drop(columns=["X", "Y"])

# IF you have no bethymetry data uncomment and use this 
datasb = {'SB_Z': [0]}  # You can add more rows if needed
BATHY = gpd.GeoDataFrame(datasb, geometry=[Point(0, 0)])


In [35]:

#OBP Varibles load in example, if using layer cake model
# This is an example of how to do this approach. You will have to do this yourself for your own AOI. Or just use a gradient. note that will require some changes in thhe function 'process_cavern_data' and xx


#Datum = read_zmap_to_geopandas('E:\\Data\\CHapter 2\\Site1\\Datum.ascii')
#Datum.rename(columns={'Z': 'Z_Datum'}, inplace=True)
#Seabed = read_zmap_to_geopandas('E:\\Data\\CHapter 2\\Site1\\Seabed.ascii')
#Seabed.rename(columns={'Z': 'Z_Seabed'}, inplace=True)
#Bunter = read_zmap_to_geopandas('E:\\Data\\CHapter 2\\Site1\\Bunter.ascii')
#Bunter.rename(columns={'Z': 'Z_Bunter'}, inplace=True)
#TopZechob = read_zmap_to_geopandas('E:\\Data\\CHapter 2\\Site1\\TopZechob.ascii')
#TopZechob.rename(columns={'Z': 'Z_TopZechob'}, inplace=True)
#Topstass = read_zmap_to_geopandas('E:\\Data\\CHapter 2\\Site1\\topz2.ascii')
#Topstass.rename(columns={'Z': 'Z_Topstass'}, inplace=True)


#dataframes = [Datum, Seabed, Bunter, TopZechob, Topstass]
#df_names = ['Datum', 'Seabed', 'Bunter', 'TopZechob', 'Topstass']
#Z_names = ['Z_Datum', 'Z_Seabed', 'Z_Bunter', 'Z_TopZechob', 'Z_Topstass']

# Create a backup of the original DataFrame
#Original_Datum = Datum.copy()

# Reset Datum to its original state
#Datum = Original_Datum.copy()

#df_layers  = Datum.copy()  # Create a new DataFrame to store the modified data

#for df, name, Z_name in zip(dataframes[1:], df_names[1:], Z_names[1:]):
    #df_layers  = nearestneighbor_obp2(df_layers, df, Z_name)
    #df_layers .rename(columns={Z_name: f'Z_{name}'}, inplace=True)


#df_layers.rename(columns={'Z': 'Z_Datum'}, inplace=True)
#df_layers

In [38]:
#Faults buffer distance (m). You can change this
Fault_buffer_dist = 250


## Make zone for suitable caverns to go into, not using on salt geoms.....This is a first filter almost

In [62]:
Suitable_Zone = make_suitable_Cavern_Zone(faults=Faults, faultsbuffer=Fault_buffer_dist, HC_Fields=HC_Fields, depo_feat=Evap_depo_feat, salt_thick=None, salt_depth=None, siteoutline=Site1)

C:\Users\c0063859\AppData\Local\Temp\ipykernel_34964\4081295444.py:35: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:32631
Right CRS: None

  diff2 = diff1.difference(HC_Fields)


In [63]:
# Test to see if it looks okay...... ours does!
Suitable_Zone.explore()

In [66]:
df_layers = 1 # If not doing layer cake model, we need to give part of the layercake model this value so it doesn't crash the function even though it's not used........ Quick and easy fix i guess, if i was a compu scientist over a geologist this wouldn't exist. I'll revise at some point

In [89]:
#Set parameters for the workflow. Some of these have weird names, due legacy programming.........(i.e i did this and forgot how to fix)


#If wanting a determintic run, you can change the std deviation or high low of the varibles here#


Params = {
    'Salt_Thick': Salt_Thick, # Zmap of salt thickness we loaded in earlier (m)
    'Salt_Depth': Salt_Depth, # Zmap of Depth to target to salt we loaded in (m)
    'Suitable_Zone_10': Suitable_Zone,# Suitable zone we made just above. or if no spaital suitable zone, just a polygon of the surface areas
    'BATHY': BATHY,
   # 'df_layers': df_layers, # This is for layer cake overburden model stuff
   # 'water_den_mean': 1023.6, # water density
   # 'water_den_std': 1,
   # 'tert_den_mean': 2390, # Tert density . eg layer 1
   # 'tert_den_std': 30,
   # 'bunter_den_mean': 2570, # Bunter density eg layer 2
   # 'bunter_den_std': 50,
   # 'Zech_top_mean': 2360, # Top Zechstein density. eg layer 3
   # 'Zech_top_std': 40,
   # 'halite_mean': 2050,
   # 'halite_std': 10,
   # 'hanging_wall_thick_mean': 103,
   # 'hanging_wall_thick_std': 0.1, #END OF LAYER CAKE OBP STUFF
    'insol_mean': 92.06, # insolubility distrabution stuff
    'insol_std_dev': 7.27, # insolubility distrabution stuff
    'geotherm_low': 32, # Geothemral gradinet high for uniform distrabution
    'geotherm_high': 52 # Geothemral gradinet high for uniform distrabution
}

In [90]:
#Example of running a single iteration. Change the depth uncertainity from 7 (%) to 0 for a no depth uncertainity run, bare in mind the geothermal and soluble unceertainities are baked into the parms and will need to be modified too
Cavern_Cluster = combined_pro(Salt_Thick, Salt_Depth, Suitable_Zone, df_layers, Params, 7)


# Note the above iteration is using varible caverns. You will need to change parameters in a few function to use fixed caverns like the industry standard
# in function 'add_cavern_data' you will need to change mminimum salt thickness for salt cavern size. so for 300 height, there is a commented line of 358.5 m thick, as this is what is needed for 300m tall caverns
# There are also commented lines for set cavern geometrys that need uncommenting if you want to use those!

0.9673168440232107
                 X             Y           Z  \
0      293904.5422  6.068358e+06  394.604235   
1      293904.5422  6.068408e+06  395.663543   
2      293904.5422  6.068458e+06  396.948914   
3      293904.5422  6.068508e+06  399.525362   
4      293904.5422  6.068558e+06  403.975600   
...            ...           ...         ...   
90129  306654.5422  6.081408e+06   97.911327   
90130  306654.5422  6.081458e+06  101.773727   
90131  306654.5422  6.081508e+06  104.124694   
90132  306654.5422  6.081558e+06  103.395820   
90133  306654.5422  6.081608e+06  100.064284   

                                       geometry        Depth  depth_uncert  
0      POINT Z (293904.542 6068357.619 407.937)  1470.662969      0.967317  
1      POINT Z (293904.542 6068407.619 409.032)  1470.662969      0.967317  
2      POINT Z (293904.542 6068457.619 410.361)  1470.662969      0.967317  
3      POINT Z (293904.542 6068507.619 413.024)  1471.153399      0.967317  
4      POINT Z (293

In [91]:
Cavern_Cluster

,X,Y,thickness,Depth,depth_uncert,SB_Z,Base,thick_avail,cav_top,Extra_hanging_wall,...,cav_vol,geometry,cav_mid_to_sb,gtg,cav_mid_temp,insolubles%,act_cav_vol,obp,H2_Mass_ton_TM,H2_E_GWh_TM
223,294104.5422,6.068308e+06,384.070831,1481.728977,0.967317,0,1865.799808,384.070831,1481.728977,0.0,...,3.241158e+06,"POLYGON ((294392.198 6068307.619, 294390.812 6...",1634.211782,47.026059,88.850539,0.827466,2.681948e+06,38.475882,41509.787983,1383.198380
235,294104.5422,6.068908e+06,410.509922,1473.866432,0.967317,0,1884.376354,410.509922,1473.866432,0.0,...,3.637455e+06,"POLYGON ((294390.003 6068907.619, 294388.629 6...",1639.870505,47.026059,89.116647,0.977483,3.555550e+06,38.609111,55180.917492,1838.750795
359,294154.5422,6.069508e+06,379.237826,1483.449737,0.967317,0,1862.687563,379.237826,1483.449737,0.0,...,3.150737e+06,"POLYGON ((294441.172 6069507.619, 294439.791 6...",1633.657105,47.026059,88.824455,0.971843,3.062022e+06,38.462823,47379.698125,1578.796830
371,294154.5422,6.070108e+06,365.939058,1481.139204,0.967317,0,1847.078261,365.939058,1481.139204,0.0,...,2.906278e+06,"POLYGON ((294438.249 6070107.619, 294436.883 6...",1625.098993,47.026059,88.422001,0.986962,2.868386e+06,38.261331,44200.164817,1472.847714
516,294204.5422,6.070708e+06,352.872638,1501.503061,0.967317,0,1854.375700,352.872638,1501.503061,0.0,...,2.672566e+06,"POLYGON ((294485.228 6070707.619, 294483.876 6...",1639.345125,47.026059,89.091940,0.890477,2.379858e+06,38.596742,36925.257355,1230.431631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87044,305854.5422,6.072958e+06,374.266591,1676.349257,0.967317,0,2050.615848,374.266591,1676.349257,0.0,...,3.058599e+06,"POLYGON ((306140.097 6072957.619, 306138.722 6...",1824.218821,47.026059,97.785821,0.909658,2.782279e+06,42.949408,46911.088777,1563.181725
87057,305854.5422,6.073608e+06,403.837661,1660.096689,0.967317,0,2063.934350,403.837661,1660.096689,0.0,...,3.579586e+06,"POLYGON ((306143.405 6073607.619, 306142.014 6...",1822.296888,47.026059,97.695440,0.950158,3.401171e+06,42.904158,57299.561302,1909.348715
87400,305904.5422,6.075758e+06,355.860100,1662.026486,0.967317,0,2017.886586,355.860100,1662.026486,0.0,...,2.725418e+06,"POLYGON ((306185.932 6075757.619, 306184.577 6...",1801.265393,47.026059,96.706412,0.851333,2.320237e+06,42.408992,38741.292279,1290.945950
87683,305954.5422,6.074958e+06,380.712984,1664.707502,0.967317,0,2045.420486,380.712984,1664.707502,0.0,...,3.178248e+06,"POLYGON ((306241.487 6074957.619, 306240.105 6...",1815.609120,47.026059,97.380941,0.901730,2.865921e+06,42.746701,48145.863854,1604.327174


In [92]:
Cavern_Cluster.explore()

In [78]:
# Can then output
Cavern_Cluster.to_file("X:/pathtostore/CAVERNS.gpkg")

### Depending on geol model size, processing time can vary from 1 min - 45 mins (Depending on data size/area extent). If using a montecarlo approach (BELOW) can take a long time...... below is a multcore code to undertake the multicore. note this will 100% all your processor cores, change the n_jobs=-1 to however many core you want to use.

In [85]:


# Set number of iterations
n_iterations = 15

# Create a list of seeds
seeds = list(range(1, n_iterations+1))

# Define the function you want to run in parallel Cavern_Cluster
run_funcs = [('Test_uncerttn0', lambda seed: combined_pro(Salt_Thick, Salt_Depth, Suitable_Zone, df_layers, Params, 7))] #NEEDS RE RUNNING


# Generate jobs for each function and seed combination
jobs = [(func_name, run_func, seed) for func_name, run_func in run_funcs for seed in seeds]

# Initialize empty lists for storing results
df_list = []
sum_list = []

# Run all jobs in parallel
#(n_jobs=-1) for all cores PC has, for basin scale may run out of ram. did with SNS..... look at pickle version below for v large datasets
results = Parallel(n_jobs= 30)(delayed(lambda func_name, run_func, seed: run_func(seed))(*job) for job in jobs)

# Iterate over results
for i, result in enumerate(results):
    # Extract function run and seed number from the job details
    func_name, _, seed = jobs[i]

    # Store the dataframe and its source details in df_list
    df_list.append({
        'func_run': func_name,
        'seed': seed,
        'df': result
    })

    # Store the sum of 'H2_E_GWh_TM' and its source details in sum_list
    sum_list.append({
        'func_run': func_name,
        'Iteration': seed,
        'Depth_Uncertainity': result['depth_uncert'].sum() / len(result),
        'Geothermal_Gradient': result['gtg'].sum() / len(result),
        'sum': result['H2_E_GWh_TM'].sum(),
        'Cavern_Number': len(result), # Modify this line if needed
        'Avg_Cav_Cap': result['H2_E_GWh_TM'].sum() / len(result)
    })

# Convert lists of dicts to dataframes for better visualization
df_results = pd.DataFrame(df_list)
sum_results = pd.DataFrame(sum_list)


In [83]:
# List of df's seed is actually iteration.
df_results
# You can call each df to its own varible as follows
df_to_look_at = df_results.loc[0, 'df']

,func_run,seed,df
0,Test_uncerttn0,1,X Y thickness ...
1,Test_uncerttn0,2,X Y thickness ...
2,Test_uncerttn0,3,X Y thickness ...
3,Test_uncerttn0,4,X Y thickness ...
4,Test_uncerttn0,5,X Y thickness ...
5,Test_uncerttn0,6,X Y thickness ...
6,Test_uncerttn0,7,X Y thickness ...
7,Test_uncerttn0,8,X Y thickness ...
8,Test_uncerttn0,9,X Y thickness ...
9,Test_uncerttn0,10,X Y thickness ...


In [86]:
sum_results

,func_run,Iteration,Depth_Uncertainity,Geothermal_Gradient,sum,Cavern_Number,Avg_Cav_Cap
0,Test_uncerttn0,1,0.938408,41.976424,444788.082781,504,882.516037
1,Test_uncerttn0,2,0.966532,36.240925,471006.117284,477,987.434208
2,Test_uncerttn0,3,1.057400,50.112048,407415.902515,358,1138.033247
3,Test_uncerttn0,4,0.967254,38.387216,468081.488530,478,979.249976
4,Test_uncerttn0,5,1.065994,42.209050,405484.292245,339,1196.118856
5,Test_uncerttn0,6,0.994273,45.384590,454702.521548,450,1010.450048
6,Test_uncerttn0,7,0.953409,45.987368,450845.246194,492,916.352126
7,Test_uncerttn0,8,0.991634,33.760953,479963.103215,451,1064.219741
8,Test_uncerttn0,9,0.939942,40.061451,452231.777542,501,902.658239
9,Test_uncerttn0,10,0.995433,44.960751,458336.518422,451,1016.267225


In [43]:


# Define colors
hist_color = "lightblue"  
line_color = "darkred"

# Create the histogram plot using Plotly
fig = go.Figure()

data = sum_results['sum'] / 1000

# Add histogram trace
fig.add_trace(go.Histogram(
    x=data,
    nbinsx=50,
    name='Samples',
    marker=dict(
        color=hist_color,
        line=dict(
            color='rgba(0, 0, 0, 1)',
            width=2
        )
    ),
    opacity=0.75
))

# KDE
kde = gaussian_kde(data)
x_range = np.linspace(min(data), max(data), 200)
kde_values = kde(x_range)

# Scale KDE values to match histogram
bin_width = (max(data) - min(data)) / 50 # bin width
scale_factor = len(data) * bin_width *1 # scale factor
kde_values_scaled = kde_values * scale_factor # scaled KDE values

# Add KDE trace
fig.add_trace(go.Scatter(
    x=x_range,
    y=kde_values_scaled,
    mode='lines',
    name='KDE',
    line=dict(
        color=line_color,
        width=2
    )
))

# Calculate percentile values
p90 = np.percentile(data, 10)
p50 = np.percentile(data, 50)
p10 = np.percentile(data, 90)

# Add text
fig.add_annotation(
    text=f'Median (p50): {p50:.2f}<br>p10: {p10:.2f}<br>p90: {p90:.2f}',
    align='left',
    showarrow=False,
    xref='paper',
    yref='paper',
    x=0.95,  # Position in the x axis (0 leftmost - 1 rightmost)
    y=0.95,  # Position in the y axis (0 bottommost - 1 topmost)
    bgcolor='white',
    bordercolor='black'
)

# Overlay the histogram
fig.update_layout(
    barmode='overlay',
    title_text='Structured Salt - Fixed Caverns: Total Energy Capacity (TWh) Montecarlo (2500)',
    title_x=0.5,
    title_font=dict(
        size=24,
        color='rgba(0, 0, 0, 1)'
    ),
    xaxis=dict(
        title='Hydrogen Capacity (TWh)',
        titlefont=dict(
            size=18,
            color='rgba(0, 0, 0, 1)'
        ),
        tickfont=dict(
            size=16,
            color='rgba(0, 0, 0, 1)'
        )
    ),
    yaxis=dict(
        title='Frequency',
        titlefont=dict(
            size=18,
            color='rgba(0, 0, 0, 1)'
        ),
        tickfont=dict(
            size=16,
            color='rgba(0, 0, 0, 1)'
        )
    ),
    template='none',
    autosize=False,
    width=1200,
    height=800
)

fig.show()

# Note these value are slightloy different to 4.4.1 section as that used layer cake OBP model, this example is using a quick and easy gradient........ Just an FYI

In [46]:
import plotly.graph_objects as go
import numpy as np
from scipy.stats import gaussian_kde

# Define colors
hist_color = "lightgreen"  
line_color = "darkred"

# Create the histogram plot using Plotly
fig = go.Figure()

data = sum_results['Cavern_Number']

# Add histogram trace
fig.add_trace(go.Histogram(
    x=data,
    nbinsx=25,
    name='Samples',
    marker=dict(
        color=hist_color,
        line=dict(
            color='rgba(0, 0, 0, 1)',
            width=2
        )
    ),
    opacity=0.75
))


fig.add_trace(go.Scatter(
    x=x_range,
    y=kde_values_scaled,
    mode='lines',
    name='KDE',
    line=dict(
        color=line_color,
        width=2
    )
))

# Calculate percentile values
p90 = np.percentile(data, 10)
p50 = np.percentile(data, 50)
p10 = np.percentile(data, 90)

# Add text
fig.add_annotation(
    text=f'Median (p50): {p50:.2f}<br>p10: {p10:.2f}<br>p90: {p90:.2f}',
    align='left',
    showarrow=False,
    xref='paper',
    yref='paper',
    x=0.85,  # Position in the x axis (0 leftmost - 1 rightmost)
    y=0.95,  # Position in the y axis (0 bottommost - 1 topmost)
    bgcolor='white',
    bordercolor='black'
)

# Overlay the histogram
fig.update_layout(
    barmode='overlay',
    title_text='Structured Salt - Fixed Caverns: Cavern Number - Montecarlo (2500)',
    title_x=0.5,
    title_font=dict(
        size=24,
        color='rgba(0, 0, 0, 1)'
    ),
    xaxis=dict(
        title='Number of Caverns',
        titlefont=dict(
            size=18,
            color='rgba(0, 0, 0, 1)'
        ),
        tickfont=dict(
            size=16,
            color='rgba(0, 0, 0, 1)'
        )
    ),
    yaxis=dict(
        title='Frequency',
        titlefont=dict(
            size=18,
            color='rgba(0, 0, 0, 1)'
        ),
        tickfont=dict(
            size=16,
            color='rgba(0, 0, 0, 1)'
        )
    ),
    template='none',
    autosize=False,
    width=1200,
    height=800
)

fig.show()


In [ ]:
# Find the geospatial results closest to the p10.50 and 90 values. Note there will be large variation with cavern placement here due to geothermal gradient and other varibles

p10 = np.percentile(sum_results['sum'], 10)
p50 = np.percentile(sum_results['sum'], 50)
p90 = np.percentile(sum_results['sum'], 90)

# Find the index of the closest value to each percentile
idx_closest_to_p10 = abs(sum_results['sum'] - p10).idxmin()
idx_closest_to_p50 = abs(sum_results['sum'] - p50).idxmin()
idx_closest_to_p90 = abs(sum_results['sum'] - p90).idxmin()

# Retrieve the DataFrame entries closest to the percentiles
df_to_look_at_p10 = df_results.loc[idx_closest_to_p10, 'df']
df_to_look_at_p50 = df_results.loc[idx_closest_to_p50, 'df']
df_to_look_at_p90 = df_results.loc[idx_closest_to_p90, 'df']

In [ ]:
# OUTPUT DATA
df_to_look_at_p10.to_file("e:\X.gpkg", driver="GPKG")
df_to_look_at_p50.to_file("e:\X.gpkg", driver="GPKG")
df_to_look_at_p90.to_file("e:\X.gpkg", driver="GPKG")

sum_results.to_csv('e:\Paper2OUTPUTMODELS\Block Level\Layered_Evaporite\VarCav_BasinWideData\Results_output_VarCav_PubDat.csv', index=False)

### If the outputs from the montecarlo are too large for your ram e.g 2500 iterations at 300mb an output (basin wide example) Then an alternate is to use pickle to save each iteration as a file to a local drive, then unpickle the iterations you are interested in

In [ ]:
#THIS IS FOR local iteration storage

#directory to save too
directory = r'F:\C0063859\Models\Layervar'  # Use a raw string for the path
if not os.path.exists(directory):
    os.makedirs(directory)

import pickle  # Import pickle library

def combined_pro(Salt_Thick, Salt_Depth, Suitable_Zone_10, df_layers, params, depth_uncert1, seed, OBP=None):
    depth_uncert2 = generate_random_percentage(depth_uncert1)
    print(depth_uncert2)
    P1 = process_salt_data(Salt_Thick, Salt_Depth, Suitable_Zone_10, params, depth_uncert2)
    
    if OBP is None:
        OBP = calculate_overburden_pressure(df_layers, params, depth_uncert2)
        
    P2 = process_cavern_data(P1, OBP, params)

    # Save the results to a pickle file in the E/Memory directory
    with open(os.path.join(directory, f'P2_results_{seed}.pkl'), 'wb') as f:
        pickle.dump(P2, f)
    
    return None

# Set number of iterations, 25 here as an example 2500 was sued in paper but takes a while
n_iterations = 25

# Create a list of seeds
#seeds = list(range(1, n_iterations+1))
seeds = list(range(1, n_iterations+1))


# Update lambda function in run_funcs to include 'seed' as an argument
run_funcs = [('Test_uncerttn0', lambda seed: combined_pro(Salt_Thick, Salt_Depth, Suitable_Zone_10_w_thick, df_layers, paramscombn, 7, seed, OBP=None))]


# Generate jobs for each function and seed combination
jobs = [(func_name, run_func, seed) for func_name, run_func in run_funcs for seed in seeds]

# Initialize empty lists for storing results
df_list = []
sum_list = []

# Run all jobs in parallel
results = Parallel(n_jobs=-1)(delayed(lambda func_name, run_func, seed: run_func(seed))(*job) for job in jobs)

# The pickle files will now be saved in the E/Memory directory.


### Too unpickle code is below

In [ ]:
# Function to aggregate results from pickle files
def aggregate_results(seeds, df_list, sum_list, directory):
    print("Starting to aggregate results...")
    
    for seed in seeds:
        filepath = os.path.join(directory, f'P2_results_{seed}.pkl')
        print(f"Reading file: {filepath}")
        
        try:
            with open(filepath, 'rb') as f:
                print("Unpickling data...")
                P2 = pickle.load(f)
                print("Successfully unpickled data.")
        except Exception as e:
            print(f"An error occurred while trying to read or unpickle the file: {e}")
            continue
        
    #Update df_list and sum_list
        print("Updating df_list and sum_list...")
        df_list.append({
            'func_run': 'Test_uncerttn0',  # or whatever function name you used
            'seed': seed,
            'df': P2
        })

        sum_value = P2['H2_E_GWh_TM'].sum()  # Modify this line if needed
        sum_list.append({
            'func_run': 'Test_uncerttn0',  # or whatever function name you used
            'seed': seed,
            'sum': sum_value,
            'Cavern_Number': len(P2),  # Number of rows in DataFrame
            'Avg_Cav_Cap': P2['H2_E_GWh_TM'].sum() / len(P2),
            'Geothermal_Grad': P2['gtg'].sum() / len(P2)# Average
        })
        
        print(f"Added data for seed {seed} to df_list and sum_list.")
        
    print("Finished aggregating results.")


In [ ]:
seeds = list(range(1, 2501))

In [ ]:
# Initialize empty lists for storing results
df_list = []
sum_list = []

# Run all jobs in paralle

# Aggregate results and populate df_list and sum_list
#aggregate_results(seeds, df_list, sum_list, 'F:\C0063859\BGS_MEMORY')
aggregate_results(seeds, df_list, sum_list, directory)

# Convert lists of dicts to dataframes for better visualization

#Same output as montecarlo note, if you just want capacity and do not care for geospaital data, you can un comment the df_results related code. Useful approach for very large data sets. I took this when doing the basin wide data. Then loaded the iterations i was ineterested in by setting the seed to their iteration number
df_results = pd.DataFrame(df_list)
sum_results = pd.DataFrame(sum_list)


In [ ]:
# Finding p10,50 and 90 from the output results

In [ ]:
df = sum_results
def closest_value(input_list, input_value):
    """
    This function finds the closest value to the input_value in input_list.
    It returns the closest value and its index.
    """
    array = np.asarray(input_list)
    idx = (np.abs(array - input_value)).argmin()
    return array[idx], idx

# Calculate the P10, P50, and P90 values
p10_value = sum_results['sum'].quantile(0.10)
p50_value = sum_results['sum'].quantile(0.50)
p90_value = sum_results['sum'].quantile(0.90)

# Find the closest values and their indices
closest_p10, idx_p10 = closest_value(df['sum'], p10_value)
closest_p50, idx_p50 = closest_value(df['sum'], p50_value)
closest_p90, idx_p90 = closest_value(df['sum'], p90_value)

# Print the results
print("P10 value:", p10_value, "| Closest value:", closest_p10, "| iteration:", df.iloc[idx_p10]['iteration'])
print("P50 value:", p50_value, "| Closest value:", closest_p50, "| iteration:", df.iloc[idx_p50]['iteration'])
print("P90 value:", p90_value, "| Closest value:", closest_p90, "| iteration:", df.iloc[idx_p90]['iteration'])
